### Create connection to the Azure data lake Gen2 storage account Container

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant-id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://olympics21-data@olympics21.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolympics",
extra_configs = configs)
  

True

In [ ]:
# verify connection
%fs
ls "/mnt/tokyoolympics"

path,name,size,modificationTime
dbfs:/mnt/tokyoolympics/raw-data/,raw-data/,0,1695023079000
dbfs:/mnt/tokyoolympics/transformed-data/,transformed-data/,0,1695023093000


### Read the csv files in the raw-data container

In [ ]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolympics/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolympics/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolympics/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolympics/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolympics/raw-data/teams.csv")

### Accessing the files

In [ ]:
# Athletes
athletes.show(5)

+-----------------+-------+-------------------+
|       PersonName|Country|         Discipline|
+-----------------+-------+-------------------+
|  AALERUD Katrine| Norway|       Cycling Road|
|      ABAD Nestor|  Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|  Italy|             Rowing|
|   ABALDE Alberto|  Spain|         Basketball|
|    ABALDE Tamara|  Spain|         Basketball|
+-----------------+-------+-------------------+
only showing top 5 rows



In [ ]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [ ]:
coaches.show(5)

+---------------+-------------+----------+-----+
|           Name|      Country|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| NULL|
|      ABE Junya|        Japan|Volleyball| NULL|
|  ABE Katsuhiko|        Japan|Basketball| NULL|
|   ADAMA Cherif|C�te d'Ivoire|  Football| NULL|
|     AGEBA Yuya|        Japan|Volleyball| NULL|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [ ]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
entriesgender.show(5)


+-------------------+------+----+-----+
|         Discipline|Female|Male|Total|
+-------------------+------+----+-----+
|     3x3 Basketball|    32|  32|   64|
|            Archery|    64|  64|  128|
|Artistic Gymnastics|    98|  98|  196|
|  Artistic Swimming|   105|   0|  105|
|          Athletics|   969|1072| 2041|
+-------------------+------+----+-----+
only showing top 5 rows



In [ ]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
medals.show(5)

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
+----+--------------------+----+------+------+-----+-------------+
only showing top 5 rows



In [ ]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [ ]:
teams.show(5)

+--------+--------------+--------------------+-----+
|TeamName|    Discipline|             Country|Event|
+--------+--------------+--------------------+-----+
| Belgium|3x3 Basketball|             Belgium|  Men|
|   China|3x3 Basketball|People's Republic...|  Men|
|   China|3x3 Basketball|People's Republic...|Women|
|  France|3x3 Basketball|              France|Women|
|   Italy|3x3 Basketball|               Italy|Women|
+--------+--------------+--------------------+-----+
only showing top 5 rows



In [ ]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



### Analysis

In [ ]:
# Find the top 10 countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team_Country","Gold").show(10)

+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|               Italy|  10|
|             Germany|  10|
|              France|  10|
|         Netherlands|  10|
+--------------------+----+
only showing top 10 rows



In [ ]:
# What is the number of male and female at the 2021 olympics

from pyspark.sql.functions import sum

entriesgender.select(sum(entriesgender.Male).alias("Total Male"), 
          sum(entriesgender.Female).alias("Total Female")
    ).show()

+----------+------------+
|Total Male|Total Female|
+----------+------------+
|      5884|        5432|
+----------+------------+



In [ ]:
# Find the top 10 countries with the highest number of athletes

top_countries_highest_athletes = athletes.groupBy("Country").count().orderBy("count", ascending=False)
top_countries_highest_athletes.show(10)

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|United States of ...|  615|
|               Japan|  586|
|           Australia|  470|
|People's Republic...|  401|
|             Germany|  400|
|              France|  377|
|              Canada|  368|
|       Great Britain|  366|
|               Italy|  356|
|               Spain|  324|
+--------------------+-----+
only showing top 10 rows



In [ ]:
# Find the top 10 most popular Disciplines(activities) at the Olympics

top_10_activities = entriesgender.orderBy("Total", ascending=False).select("Discipline","Total").show(10)

+------------+-----+
|  Discipline|Total|
+------------+-----+
|   Athletics| 2041|
|    Swimming|  779|
|    Football|  608|
|      Rowing|  522|
|        Judo|  393|
|      Hockey|  384|
|    Shooting|  356|
|     Sailing|  350|
|    Handball|  336|
|Rugby Sevens|  297|
+------------+-----+
only showing top 10 rows



### Load data into the transformed-data container on Azure data lake Gen2 storage account

In [ ]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympics/transformed-data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolympics/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolympics/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolympics/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolympics/transformed-data/teams")